In [45]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from dotenv import load_dotenv
import os
import csv
import time
import datetime

In [34]:
# Open the browser
options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": f"{os.getcwd()}/downloads"
}
options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

/var/folders/hq/ypf9892n1s94fwgm0p6k6k8m0000gn/T/ipykernel_35540/3579103318.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
/var/folders/hq/ypf9892n1s94fwgm0p6k6k8m0000gn/T/ipykernel_35540/3579103318.py:7: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)


In [35]:
# Get username and password from .env file
load_dotenv()
USER = os.getenv("USERNAME")
PASS = os.getenv("PASS")
COURSE_NO = os.getenv("COURSE_NO")
ASSIGNMENT_NO = os.getenv("ASSIGNMENT_NO")
BASE_URL = "https://www.gradescope.com"

In [36]:
# Log in
login_url = f"{BASE_URL}/login"
driver.get(login_url)
username_input = driver.find_element("id", "session_email")
password_input = driver.find_element("id", "session_password")
submit = driver.find_element("name", "commit")
username_input.send_keys(USER)
password_input.send_keys(PASS)
submit.click()

In [40]:
# Go to assignment "review grades" page
download_url = f"{BASE_URL}/courses/{COURSE_NO}/assignments/{ASSIGNMENT_NO}/scores.csv"
driver.get(download_url)
time.sleep(2)
try:
    os.remove(f"{os.getcwd()}/downloads/csv/scores.csv")
except FileNotFoundError:
    pass
files = os.listdir("downloads/")
for file in files:
    if ".csv" in file:
        os.rename(f"downloads/{file}", f"downloads/csv/scores.csv")

In [50]:
# Read CSV to get URLS, names, and TFs
tfs = {}
with open("last_sweep.txt", "r") as f:
    last_sweep = f.read()

with open("downloads/csv/scores.csv", "r") as f:
    reader = csv.DictReader(f)
    most_recent = last_sweep
    print(most_recent)
    for row in reader:
        tf = row["Sections"]
        if tf not in tfs:
            tfs[tf] = []
        if row["Submission Time"]:
            if row["Submission Time"] > most_recent:
                most_recent = row["Submission Time"]
            tfs[tf].append({
                "student": row["Name"],
                "link": f"{BASE_URL}/courses/{COURSE_NO}/assignments/{ASSIGNMENT_NO}/submissions/{row    ['Submission ID']}.zip"
            })

with open("last_sweep.txt", "w") as f:
    f.write(most_recent)

2022-11-05 21:29:08 -0400


{'Kamryn Ohly': [{'student': 'Zach Elan Abrams',
   'link': 'https://www.gradescope.com/courses/411020/assignments/2140454/submissions/145394701.zip'},
  {'student': 'William Zhang',
   'link': 'https://www.gradescope.com/courses/411020/assignments/2140454/submissions/146186858.zip'},
  {'student': 'Joey Philippe Bejjani',
   'link': 'https://www.gradescope.com/courses/411020/assignments/2140454/submissions/146422254.zip'}],
 'Nick DeSanctis': [{'student': 'Cameron Hamby',
   'link': 'https://www.gradescope.com/courses/411020/assignments/2140454/submissions/145560078.zip'},
  {'student': 'Darren Dinh-Anh-Kien Tran',
   'link': 'https://www.gradescope.com/courses/411020/assignments/2140454/submissions/146721632.zip'}],
 'Michael Hu': [{'student': 'Phoebe Zhang',
   'link': 'https://www.gradescope.com/courses/411020/assignments/2140454/submissions/145654106.zip'},
  {'student': 'Michelle Jia Chang',
   'link': 'https://www.gradescope.com/courses/411020/assignments/2140454/submissions/145